data cleaning for dataset from glassdoor

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("rashikrahmanpritom/data-science-job-posting-on-glassdoor")

# print("Path to dataset files:", path)

C:\Users\manir\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.53M/1.53M [00:00<00:00, 8.61MB/s]

Extracting files...
Path to dataset files: C:\Users\manir\.cache\kagglehub\datasets\rashikrahmanpritom\data-science-job-posting-on-glassdoor\versions\1


In [3]:
import pandas as pd

# Replace 'path_to_your_file.csv' with the actual file path
df = pd.read_csv(r'C:\Users\manir\.cache\kagglehub\datasets\rashikrahmanpritom\data-science-job-posting-on-glassdoor\versions\1\Uncleaned_DS_jobs.csv')


In [4]:
# Display the first few rows to understand the structure of the dataset
print(df.head())


   index          Job Title               Salary Estimate  \
0      0  Sr Data Scientist  $137K-$171K (Glassdoor est.)   
1      1     Data Scientist  $137K-$171K (Glassdoor est.)   
2      2     Data Scientist  $137K-$171K (Glassdoor est.)   
3      3     Data Scientist  $137K-$171K (Glassdoor est.)   
4      4     Data Scientist  $137K-$171K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

              Company Name       Location            Headquarters  \
0         Healthfirst\n3.1   New York, NY            New York, NY   
1             ManTech\n4.2  Chantilly, VA             Herndon, VA   
2      Analysis Group\n3.8     Boston, MA

In [15]:
# Remove any text in parentheses
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'\(.*\)', '', regex=True)

# Remove 'K' and '$' symbols, then split the range into lower and upper bounds
salary_split = df['Salary Estimate'].str.replace('K', '').str.replace('$', '').str.split('-')

# Convert lower and upper bounds to integers and calculate the average
df['min_salary'] = salary_split.str[0].astype(float)
df['max_salary'] = salary_split.str[1].astype(float)
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2



In [17]:
# Replace 'job_description' with 'Job Description'
df['python_skill'] = df['Job Description'].str.contains('Python', case=False, na=False)
df['sql_skill'] = df['Job Description'].str.contains('SQL', case=False, na=False)



In [19]:
# Replace 'company_name' with 'Company Name'
df['Company Name'] = df['Company Name'].str.replace(r'\d+', '', regex=True)



In [22]:
# Split 'Location' column into two separate columns: 'city' and 'state'
location_split = df['Location'].str.split(', ', expand=True)

# Assign the results to new columns, handling cases where there's only one part
df['city'] = location_split[0]
df['state'] = location_split[1]


In [23]:
print(df.head())


   index          Job Title Salary Estimate  \
0      0  Sr Data Scientist      137K-171K    
1      1     Data Scientist      137K-171K    
2      2     Data Scientist      137K-171K    
3      3     Data Scientist      137K-171K    
4      4     Data Scientist      137K-171K    

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

            Company Name       Location            Headquarters  \
0         Healthfirst\n.   New York, NY            New York, NY   
1             ManTech\n.  Chantilly, VA             Herndon, VA   
2      Analysis Group\n.     Boston, MA              Boston, MA   
3             INFICON\n.     Newton, MA  Bad Ragaz, Switzerland 

Dataset Cleaning Summary
The dataset has been cleaned and transformed based on the initial requirements. Below are the details of the transformations applied:

1. Salary Conversion
Extracted minimum and maximum salary values from the Salary Estimate column.
Created new columns:
min_salary: Minimum salary from the estimate.
max_salary: Maximum salary from the estimate.
avg_salary: Calculated average salary.
2. Skill Extraction
Added new columns based on keyword presence in the Job Description column:
python_skill: Indicates if "Python" is mentioned.
sql_skill: Indicates if "SQL" is mentioned.
3. Company Name Cleanup
Removed unwanted trailing characters (e.g., \n.) from Company Name for consistency and cleaner presentation.
4. Location Split
Split the Location column into separate city and state columns for more granular analysis.